In [1]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from metrics import *
from datagenerators import *
from architectures.ResidualUnet import *
from tensorflow_addons import layers

from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_res_unet(n_channels=3, ker_init='he_normal')
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 instance_normalization (Instan  (None, 128, 128, 16  32         ['conv2d[0][0]']                 
 ceNormalization)               )                                                             

                                                                                                  
 conv2d_8 (Conv2D)              (None, 16, 16, 128)  73856       ['max_pooling2d_2[0][0]']        
                                                                                                  
 instance_normalization_8 (Inst  (None, 16, 16, 128)  256        ['conv2d_8[0][0]']               
 anceNormalization)                                                                               
                                                                                                  
 activation_8 (Activation)      (None, 16, 16, 128)  0           ['instance_normalization_8[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_9 (Conv2D)              (None, 16, 16, 128)  147584      ['activation_8[0][0]']           
          

                                                                                                  
 activation_16 (Activation)     (None, 16, 16, 128)  0           ['instance_normalization_16[0][0]
                                                                 ']                               
                                                                                                  
 activation_15 (Activation)     (None, 16, 16, 128)  0           ['instance_normalization_15[0][0]
                                                                 ']                               
                                                                                                  
 add_4 (Add)                    (None, 16, 16, 128)  0           ['activation_16[0][0]',          
                                                                  'activation_15[0][0]']          
                                                                                                  
 up_sampli

 instance_normalization_23 (Ins  (None, 128, 128, 16  32         ['conv2d_23[0][0]']              
 tanceNormalization)            )                                                                 
                                                                                                  
 activation_23 (Activation)     (None, 128, 128, 16  0           ['instance_normalization_23[0][0]
                                )                                ']                               
                                                                                                  
 conv2d_25 (Conv2D)             (None, 128, 128, 16  784         ['concatenate_3[0][0]']          
                                )                                                                 
                                                                                                  
 conv2d_24 (Conv2D)             (None, 128, 128, 16  2320        ['activation_23[0][0]']          
          

NameError: name 'dice_updated' is not defined

In [3]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=336, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14)

In [4]:
training_generator = DataGenerator2D(train_ids)
valid_generator = DataGenerator2D(val_ids)
test_generator = DataGenerator2D(test_ids)

In [5]:
csv_logger = CSVLogger('Logs/training/resU_net_2d_tessst.log', separator=',', append=False)

eval_csv_logger = CSVLogger('Logs/testing/resU_net_2d_tessst.log', separator=',', append=False)


callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0000001, verbose=1),
        csv_logger
    ]

In [6]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=10,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/2D/resU_net_2d_tessst.h5")

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

In [ ]:
valid_csv_logger = CSVLogger('Logs/validation/resU_net_2d_tessst.log', separator=',', append=False)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)

In [ ]:
# model = keras.models.load_model('Models/2D/resU_net_2d_300data.h5', 
#                                    custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
#                                                    "dice_coef": dice_coef,
#                                                    "dice_coef_necrotic": dice_coef_necrotic,
#                                                    "dice_coef_edema": dice_coef_edema,
#                                                    "dice_coef_enhancing": dice_coef_enhancing
#                                                   }, compile=False)